In [ ]:
import sys
import time
import pandas as pd
from sklearn import model_selection
from sklearn import linear_model, metrics, cluster
from xgboost import XGBRegressor
from matplotlib import pyplot as plt
import scipy
import os
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.feature_selection import *
from sklearn.model_selection import GridSearchCV, KFold

start_time = time.time()
n_cv = 5
n_jobs = 6
random_state = 3111696

#dataname = "Galaxy214-[Zibotentan.csv].tabular"
#datasets = ["Galaxy215-[rTRAIL.csv].tabular"]
datasets = ["Galaxy214-[Zibotentan.csv].tabular"]

regressor = Pipeline([
  ('preprocessing', preprocessing.MinMaxScaler()),
  #('feature_selection', SelectKBest(f_regression)),
  ('regression', XGBRegressor())
])

parameters = [{ \
    #'feature_selection__k': [10, 100, 1000, 10000], \
    #'regression__max_depth': [1, 2, 3, 4, 5], \
    #'regression__n_estimators': [10, 50, 100, 200], \
    #'regression__booster': ['gbtree', 'gblinear', 'dart'], \
    'regression__random_state': [random_state] \
}]

print('Training started...')
dataset_accuracies = list()
r2_scores = list()
for d_set in datasets:
    print("Processing dataset: %s" % d_set)
    data_path = "depmap/" + d_set
    df = pd.read_csv(data_path, sep="\t")
    columns = list(df.columns.values)
    label = df[columns[-1]].copy()
    data = df.drop(columns[-1], axis=1)
    optimized_regressor = GridSearchCV(regressor, parameters, \
                                       cv=KFold(n_splits=n_cv, shuffle=True, random_state=random_state), \
                                       error_score=0, scoring='r2', verbose=True, n_jobs=n_jobs, \
                                       pre_dispatch="1*n_jobs")
    optimized_regressor.fit(data, label)
    best_regressor = optimized_regressor.best_estimator_
    best_result = optimized_regressor.cv_results_
    print(optimized_regressor.best_params_)
    best_score = optimized_regressor.best_score_
    r2_scores.append(best_score)
    print(best_score)
    print("Finished dataset: %s" % d_set)
    print("------------------------------------------------------------")

print('Training finished')
print("Mean R2 square: \n", np.mean(r2_scores))
end_time = time.time()
print('Total time taken: %d seconds' % int(end_time - start_time))

Training started...
Processing dataset: Galaxy214-[Zibotentan.csv].tabular
(610, 48117)
(610,)
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
